In [12]:
from PIL import ImageTk
import PIL
# from tkinter import *
from tkinter.filedialog import  askdirectory
from tkinter import ttk
import tkinter as tk
import os

In [13]:
import pydicom
import numpy as np
from skimage.transform import resize
from PIL import Image

from skimage import measure
from skimage.transform import resize

from tensorflow import keras
import cv2
from matplotlib import pyplot as plt
import matplotlib.patches as patches

In [14]:
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=4):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
    # output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**(depth))(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
model = create_network(input_size=256, channels=32, n_blocks=2, depth=4)
model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])
# print(model.summary())
model.load_weights('C:/Users/sz/jupyter_notebook/multi_layer_best_modelResNetLoss.h5')

In [17]:
#导入tk模块
top =  tk.Tk() # Toplevel() 
#初始化Tk
top.title('肺部不透明区域识别系统')
#标题显示为label test
label = tk.Label(top,text = '') # , text = 'this is my first label'
top.geometry('480x480')

top.resizable(width=False, height=False)  # 固定长宽不可拉伸
#创建一个label，它属于top窗口，文本显示内容为.....

path = tk.StringVar()
var = tk.StringVar()

# path_for_jpg = "C:/Users/Administrator/PycharmProjects/pythongui/"
path_for_jpg = " "
xialaliebiaozhi = []
# im2 = PIL.Image.open("D:\\YJS\\first.jpg") # 主界面
im2 = PIL.Image.open("2.jpg")
img2 = ImageTk.PhotoImage(im2)
label = tk.Label(top,image = img2)

def change(img_name_):
    # print("img_name_",img_name_)
    img_temp = pydicom.dcmread(img_name_).pixel_array # 读取dcm
    img_temp = np.array(img_temp)
    img_temp = resize(img_temp, (256, 256), mode='reflect')
    a=img_temp*255.
    b = Image.fromarray(a)
    img_tk = ImageTk.PhotoImage(b)
    label.configure(image=img_tk)
    label.image = img_tk
    var.set('')
    
def process(img_name_):
    print("img_name_",img_name_)
    # img_temp = PIL.Image.open(img_name_)
    img_temp = pydicom.dcmread(img_name_).pixel_array # 读取dcm
    img_temp = np.array(img_temp)
    img_temp = resize(img_temp, (256, 256), mode='reflect')
    
    img = np.expand_dims(img_temp, -1)
    img_4 = img.reshape(1,256,256,1)
    imgs = img_4
    preds = model.predict(imgs)

    img = imgs[0]
    pred = preds[0]
    comp = pred[:, :, 0] > 0.1

    comp = measure.label(comp)
    # predictionString = ''
    i = 0
    for region in measure.regionprops(comp):  
        y, x, y2, x2 = region.bbox
        height = y2 - y
        width = x2 - x
        i = i+1
        # print("----------------",i)
        # print("num:",measure.regionprops(comp))
        # print(x,y,x2,y2)
        
        cv2.rectangle(img[:,:,0],(x,y),(x2,y2),(0,255),2)
    print("how many regions:",i)
    img_temp = img.reshape(256,256)
    
    a=img_temp*255.
    b = Image.fromarray(a)
    img_tk = ImageTk.PhotoImage(b)
    label.configure(image=img_tk)
    label.image = img_tk
    var.set("没有不透明区域")
    if i >= 1:
        var.set("不透明区域有{}个".format(i))
    
def select_path():
    global path_for_jpg
    path_ = askdirectory()
    path.set(path_)
    path_for_jpg = path_+"/"
    print("path is :" + str(path_for_jpg).replace(' ', ''))

def find_jpg(path1):
    print("读取文件")
    global xialaliebiaozhi
    f = []
    for root, dirs, files in os.walk(path1):
        for file in files:
            if os.path.splitext(file)[1] == '.dcm':
                t = os.path.splitext(file)
                s = str(t[0])+str(t[1])
                print(s)  # 打印所有py格式的文件名
                f.append(s)  # 将所有的文件名添加到F列表中
    xialaliebiaozhi = f
    numberChosen['values'] = xialaliebiaozhi  # 设置下拉列表的值
    

# label2 = tk.Label(top,width=25, text = '目标路径：') # 文本标签
entry_dir = tk.Entry(top, width=25, textvariable = path) # 输入框
button_select_dir = tk.Button(top, text = "路径选择", command = select_path)
button_get_file_in_dir = tk.Button(top, text = "读取文件", command = lambda:find_jpg(path1=path_for_jpg))

# 创建一个下拉列表
number = tk.StringVar()
numberChosen = ttk.Combobox(top, width=25, textvariable=number)
numberChosen['values'] = ["..."]    # 设置下拉列表的值

numberChosen.current(0)    # 设置下拉列表默认显示的值，0为 numberChosen['values'] 的下标值
button = tk.Button(top, text = "显示图片", command = lambda:change(img_name_=path_for_jpg+numberChosen.get()))

numberChosen.current(0)    # 设置下拉列表默认显示的值，0为 numberChosen['values'] 的下标值
button_pro = tk.Button(top, text = "区域预测",command = lambda:process(img_name_=path_for_jpg+numberChosen.get()) )
entry_dir2 = tk.Entry(top, width=13, state = 'readonly', textvariable = var) # 输入框


label.pack()
# label2.pack()

button_select_dir.pack()
entry_dir.pack()
button_get_file_in_dir.pack()
numberChosen.pack()
button.pack()
button_pro.pack()
entry_dir2.pack()

top.mainloop()
